### **CacheBackedEmbeddings**  

`CacheBackedEmbeddings`는 임베딩을 캐싱하여 불필요한 **재계산을 방지**하는 도구

키-값 저장소를 활용하여 동일한 텍스트의 임베딩을 다시 계산하지 않고, 캐시(임시저장소)에서 바로 불러올 수 있도록 함.  

---

#### **🧐 왜 필요할까?**  

✔ **성능 최적화** – 임베딩을 캐싱하여 불필요한 계산을 줄이고 속도를 향상함.  
✔ **비용 절감** – API 기반 임베딩 서비스 사용 시 반복 계산을 줄여 비용을 절감할 수 있음.  
✔ **일관성 유지** – 동일한 텍스트에 대해 항상 동일한 임베딩 값을 제공함.  

---

#### **🔍 어떻게 작동할까?**  

1️⃣ **텍스트를 해시(hash)하여 고유 키 생성**  
2️⃣ **캐시에 해당 키가 있으면 저장된 임베딩 반환**  
3️⃣ **없으면 `underlying_embeddings`를 사용하여 계산 후 캐시에 저장**  

📌 **초기화 방법: `from_bytes_store`**  

- `underlying_embeddings`: 임베딩을 수행하는 embedder 모델  
- `document_embedding_cache`: 문서 임베딩을 저장하는 `ByteStore`  
- `namespace` (선택 사항): 임베딩 캐시의 충돌 방지를 위해 사용됨  

---

#### **🛠 어디에 활용할 수 있을까?**  

✅ **대량의 문서 분석** – 동일한 텍스트의 반복 분석 시 캐싱을 활용하여 속도를 높임.  
✅ **AI 및 NLP 시스템** – AI 모델이 동일한 입력을 반복적으로 처리할 때 성능 향상.  
✅ **검색 및 추천 시스템** – 임베딩 기반 검색에서 연산 비용 절감.  

---

#### **🎯 한 줄 요약**  

**CacheBackedEmbeddings**는 임베딩을 캐싱하여 **성능을 최적화하고 불필요한 재계산을 방지하는 도구**임. 빠르고 효율적인 NLP 및 AI 시스템 구축에 도움을 줌. 🚀

### **LocalFileStore 에서 임베딩 사용 (영구 보관)**

- 로컬 파일 시스템을 사용하여 임베딩을 저장(파일로 만들어서 보관)
- FAISS 벡터 스토어를 사용하여 검색

In [6]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# OpenAI 임베딩
embedding = OpenAIEmbeddings(model = 'text-embedding-3-small')

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings = embedding,
    document_embedding_cache = store,
    namespace = embedding.model
    
)

# store에서 키 조회
list(store.yield_keys())

[]

1. 문서로드
2. 청크 분할
3. 청크 임베딩
4. 벡터 저장소에 업로드

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(raw_documents)

CPU times: total: 0 ns
Wall time: 0 ns


%%time 

셀 전체 코드의 실행시간 상단에 쓰면됨 

%time 
한줄 사용 변수 앞에 

In [ ]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 0 ns
Wall time: 0 ns


### **`InmemoryByteStore` 사용 (비영구적)**

다른 `ByteStore`를 사용하기 위해서는 `CacheBackedEmbeddings`를 생성할 때 해당 `ByteStore`를 사용.

메모리에 일시적으로 저장함

In [17]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)